In [4]:
library(caret)
library(kernlab)
library(foreign)
library(mlbench)
library(rpart)
library(rpart.plot)
library(ipred)
library(randomForest)
library(gbm)
library(nnet)
library(pROC)
library(ggplot2)
library(doParallel)
registerDoParallel(4)
getDoParWorkers()

ERROR: ignored

In [ ]:
install.packages("caret", dependencies = c("Depends", "Suggests"))
install.packages('kernlab')
install.packages('foreign')
install.packages('mlbench')
install.packages('rpart')
install.packages('rpart.plot')
install.packages('ipred')
install.packages('randomForest')
install.packages('gbm')
install.packages('nnet')
install.packages('pROC')
install.packages('ggplot2')
install.packages('doParallel')

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Warning message:
“dependencies ‘R2wd’, ‘graph’, ‘sylly.de’, ‘sylly.es’, ‘RTCGA.rnaseq’, ‘globaltest’, ‘MassSpecWavelet’, ‘texPreview’, ‘marray’, ‘affy’, ‘Biobase’, ‘limma’, ‘Rgraphviz’, ‘RandomFields’, ‘RandomFieldsUtils’, ‘starsdata’, ‘cairoDevice’, ‘RGtk2’, ‘glmmADMB’, ‘rnaturalearthhires’, ‘mvPot’, ‘Biostrings’, ‘seqLogo’, ‘Rcampdf’, ‘tm.lexicon.GeneralInquirer’, ‘cmdstanr’, ‘dblog’, ‘lavaan.survey’, ‘BiocVersion’, ‘ExactData’, ‘ComplexHeatmap’, ‘mail’, ‘M3C’, ‘spDataLarge’, ‘RDCOMClient’, ‘gurobi’, ‘rrelaxiv’, ‘taxidata’, ‘extraTrees’, ‘genefilter’, ‘sva’, ‘corpus.JSS.papers’, ‘argon2’ are not available”
also installing the dependencies ‘Rmosek’, ‘REBayes’, ‘getPass’, ‘ashr’, ‘git2r’, ‘asciicast’, ‘shinyjs’, ‘prettyGraphs’, ‘rdd’, ‘ncvreg’, ‘gistr’, ‘filelock’, ‘pingr’, ‘pkgcache’, ‘pkgdepends’, ‘pkgsearch’, ‘fastNaiveBayes’, ‘naivebayes’, ‘ExPosition’, ‘svs’, ‘huge’, ‘LDAvis’, ‘rsvd’, ‘udpipe’, ‘OAI

In [1]:
url="https://raw.githubusercontent.com/wujinja-cgu/Model-Selection-By-Caret-Package/main/sepsis%20data.csv"
mydata=read.csv(url,header=TRUE,sep=",")
dim(mydata)

[1] 555   7

In [2]:
mydata$death=as.factor(mydata$death)
mydata1=mydata[which(mydata$death=="Survival"),]
mydata2=mydata[which(mydata$death=="Death"),]
index1=sample(dim(mydata1)[1],nrow(mydata1)*0.25)
index2=sample(dim(mydata2)[1],nrow(mydata2)*0.25)
testing=rbind(mydata1[index1,],mydata2[index2,])
training=rbind(mydata1[-index1,],mydata2[-index2,])
dim(training)
dim(testing)

[1] 417   7

[1] 138   7

In [3]:
fitControl=trainControl(method="repeatedcv",
            number=3,
            repeats=2,
            classProbs=TRUE,
            savePredictions = TRUE,
            verboseIter = TRUE,
            summaryFunction=twoClassSummary,
            search="random")

ERROR: ignored

In [11]:
model_caret=caret::modelLookup()
head(model_caret)
dim(model_caret)

,model,parameter,label,forReg,forClass,probModel
,<chr>,<chr>,<chr>,<lgl>,<lgl>,<lgl>
1,ada,iter,#Trees,FALSE,TRUE,TRUE
2,ada,maxdepth,Max Tree Depth,FALSE,TRUE,TRUE
3,ada,nu,Learning Rate,FALSE,TRUE,TRUE
4,AdaBag,mfinal,#Trees,FALSE,TRUE,TRUE
5,AdaBag,maxdepth,Max Tree Depth,FALSE,TRUE,TRUE
9,adaboost,nIter,#Trees,FALSE,TRUE,TRUE


[1] 511   6

In [12]:
model_caret_forclass=model_caret[which(model_caret$forClass=="TRUE"),]
head(model_caret_forclass)
nrow(model_caret_forclass)

,model,parameter,label,forReg,forClass,probModel
,<chr>,<chr>,<chr>,<lgl>,<lgl>,<lgl>
1,ada,iter,#Trees,FALSE,TRUE,TRUE
2,ada,maxdepth,Max Tree Depth,FALSE,TRUE,TRUE
3,ada,nu,Learning Rate,FALSE,TRUE,TRUE
4,AdaBag,mfinal,#Trees,FALSE,TRUE,TRUE
5,AdaBag,maxdepth,Max Tree Depth,FALSE,TRUE,TRUE
9,adaboost,nIter,#Trees,FALSE,TRUE,TRUE


[1] 431

In [13]:
model_caret_forreg=model_caret[which(model_caret$forReg=="TRUE"),]
head(model_caret_forreg)
nrow(model_caret_forreg)

,model,parameter,label,forReg,forClass,probModel
,<chr>,<chr>,<chr>,<lgl>,<lgl>,<lgl>
12,ANFIS,num.labels,#Fuzzy Terms,TRUE,FALSE,FALSE
13,ANFIS,max.iter,Max. Iterations,TRUE,FALSE,FALSE
14,avNNet,size,#Hidden Units,TRUE,TRUE,TRUE
15,avNNet,decay,Weight Decay,TRUE,TRUE,TRUE
16,avNNet,bag,Bagging,TRUE,TRUE,TRUE
20,bag,vars,#Randomly Selected Predictors,TRUE,TRUE,TRUE


[1] 296

In [14]:
modellist2=unique(unlist(model_caret_forclass$model))
modellist2
modellist1=modellist2[-c(46:49)]
AUC_train1=array(0,length(modellist1))
AUC_test1=array(0,length(modellist1))
length(modellist1)
modellist1

[1] "ada"                 "AdaBag"              "adaboost"           
  [4] "AdaBoost.M1"         "amdai"               "avNNet"             
  [7] "awnb"                "awtan"               "bag"                
 [10] "bagEarth"            "bagEarthGCV"         "bagFDA"             
 [13] "bagFDAGCV"           "bam"                 "bartMachine"        
 [16] "bayesglm"            "binda"               "blackboost"         
 [19] "BstLm"               "bstSm"               "bstTree"            
 [22] "C5.0"                "C5.0Cost"            "C5.0Rules"          
 [25] "C5.0Tree"            "cforest"             "chaid"              
 [28] "CSimca"              "ctree"               "ctree2"             
 [31] "dda"                 "deepboost"           "dnn"                
 [34] "dwdLinear"           "dwdPoly"             "dwdRadial"          
 [37] "earth"               "elm"                 "evtree"             
 [40] "extraTrees"          "fda"                 "FH.GBML"            
 [43] "FRBCS.CHI"           "FRBCS.W"             "gam"                
 [46] "gamboost"            "gamLoess"            "gamSpline"          
 [49] "gaussprLinear"       "gaussprPoly"         "gaussprRadial"      
 [52] "gbm"                 "gbm_h2o"             "gcvEarth"           
 [55] "glm"                 "glmboost"            "glmnet"             
 [58] "glmnet_h2o"          "glmStepAIC"          "gpls"               
 [61] "hda"                 "hdda"                "hdrda"              
 [64] "J48"                 "JRip"                "kernelpls"          
 [67] "kknn"                "knn"                 "lda"                
 [70] "lda2"                "Linda"               "LMT"                
 [73] "loclda"              "logicBag"            "LogitBoost"         
 [76] "logreg"              "lssvmLinear"         "lssvmPoly"          
 [79] "lssvmRadial"         "lvq"                 "manb"               
 [82] "mda"                 "Mlda"                "mlp"                
 [85] "mlpKerasDecay"       "mlpKerasDecayCost"   "mlpKerasDropout"    
 [88] "mlpKerasDropoutCost" "mlpML"               "mlpSGD"             
 [91] "mlpWeightDecay"      "mlpWeightDecayML"    "monmlp"             
 [94] "msaenet"             "multinom"            "mxnet"              
 [97] "mxnetAdam"           "naive_bayes"         "nb"                 
[100] "nbDiscrete"          "nbSearch"            "nnet"               
[103] "nodeHarvest"         "null"                "OneR"               
[106] "ordinalNet"          "ordinalRF"           "ORFlog"             
[109] "ORFpls"              "ORFridge"            "ORFsvm"             
[112] "ownn"                "pam"                 "parRF"              
[115] "PART"                "partDSA"             "pcaNNet"            
[118] "pda"                 "pda2"                "PenalizedLDA"       
[121] "plr"                 "pls"                 "plsRglm"            
[124] "polr"                "pre"                 "PRIM"               
[127] "protoclass"          "qda"                 "QdaCov"             
[130] "randomGLM"           "ranger"              "rbf"                
[133] "rbfDDA"              "Rborist"             "rda"                
[136] "regLogistic"         "rf"                  "rFerns"             
[139] "RFlda"               "rfRules"             "rlda"               
[142] "rmda"                "rocc"                "rotationForest"     
[145] "rotationForestCp"    "rpart"               "rpart1SE"           
[148] "rpart2"              "rpartCost"           "rpartScore"         
[151] "RRF"                 "RRFglobal"           "rrlda"              
[154] "RSimca"              "sda"                 "sdwd"               
[157] "simpls"              "SLAVE"               "slda"               
[160] "smda"                "snn"                 "sparseLDA"          
[163] "spls"                "stepLDA"             "stepQDA"            
[166] "svmBoundrangeString" "svmExpoString

[1] 187

[1] "ada"                 "AdaBag"              "adaboost"           
  [4] "AdaBoost.M1"         "amdai"               "avNNet"             
  [7] "awnb"                "awtan"               "bag"                
 [10] "bagEarth"            "bagEarthGCV"         "bagFDA"             
 [13] "bagFDAGCV"           "bam"                 "bartMachine"        
 [16] "bayesglm"            "binda"               "blackboost"         
 [19] "BstLm"               "bstSm"               "bstTree"            
 [22] "C5.0"                "C5.0Cost"            "C5.0Rules"          
 [25] "C5.0Tree"            "cforest"             "chaid"              
 [28] "CSimca"              "ctree"               "ctree2"             
 [31] "dda"                 "deepboost"           "dnn"                
 [34] "dwdLinear"           "dwdPoly"             "dwdRadial"          
 [37] "earth"               "elm"                 "evtree"             
 [40] "extraTrees"          "fda"                 "FH.GBML"            
 [43] "FRBCS.CHI"           "FRBCS.W"             "gam"                
 [46] "gaussprPoly"         "gaussprRadial"       "gbm"                
 [49] "gbm_h2o"             "gcvEarth"            "glm"                
 [52] "glmboost"            "glmnet"              "glmnet_h2o"         
 [55] "glmStepAIC"          "gpls"                "hda"                
 [58] "hdda"                "hdrda"               "J48"                
 [61] "JRip"                "kernelpls"           "kknn"               
 [64] "knn"                 "lda"                 "lda2"               
 [67] "Linda"               "LMT"                 "loclda"             
 [70] "logicBag"            "LogitBoost"          "logreg"             
 [73] "lssvmLinear"         "lssvmPoly"           "lssvmRadial"        
 [76] "lvq"                 "manb"                "mda"                
 [79] "Mlda"                "mlp"                 "mlpKerasDecay"      
 [82] "mlpKerasDecayCost"   "mlpKerasDropout"     "mlpKerasDropoutCost"
 [85] "mlpML"               "mlpSGD"              "mlpWeightDecay"     
 [88] "mlpWeightDecayML"    "monmlp"              "msaenet"            
 [91] "multinom"            "mxnet"               "mxnetAdam"          
 [94] "naive_bayes"         "nb"                  "nbDiscrete"         
 [97] "nbSearch"            "nnet"                "nodeHarvest"        
[100] "null"                "OneR"                "ordinalNet"         
[103] "ordinalRF"           "ORFlog"              "ORFpls"             
[106] "ORFridge"            "ORFsvm"              "ownn"               
[109] "pam"                 "parRF"               "PART"               
[112] "partDSA"             "pcaNNet"             "pda"                
[115] "pda2"                "PenalizedLDA"        "plr"                
[118] "pls"                 "plsRglm"             "polr"               
[121] "pre"                 "PRIM"                "protoclass"         
[124] "qda"                 "QdaCov"              "randomGLM"          
[127] "ranger"              "rbf"                 "rbfDDA"             
[130] "Rborist"             "rda"                 "regLogistic"        
[133] "rf"                  "rFerns"              "RFlda"              
[136] "rfRules"             "rlda"                "rmda"               
[139] "rocc"                "rotationForest"      "rotationForestCp"   
[142] "rpart"               "rpart1SE"            "rpart2"             
[145] "rpartCost"           "rpartScore"          "RRF"                
[148] "RRFglobal"           "rrlda"               "RSimca"             
[151] "sda"                 "sdwd"                "simpls"             
[154] "SLAVE"               "slda"                "smda"               
[157] "snn"                 "sparseLDA"           "spls"               
[160] "stepLDA"             "stepQDA"             "svmBoundrangeString"
[163] "svmExpoString"       "svmLinear"           "svmLinear2"         
[166] "svmLinear3"          "svmLinearWeig

In [15]:
for (i in 1:length(modellist1)){
    skip_to_next_error <- FALSE
    cat("Running method:", modellist1[i], "\n")
    tryCatch(modelfit<-train(death~.,
                 data=training,
                 method=modellist1[i],
                 preProc=c("center","scale"),
                 metric="ROC",
                 tuneLength=3,
                 trControl=fitControl),
                 error=function(e){skip_to_next_error <<- TRUE})
    if(skip_to_next_error) {next}
    predictions_train=predict(modelfit,newdata=training)
    predictions_test=predict(modelfit,newdata=testing)
    predictions_train
    predictions_test
    confusionMatrix(predict(modelfit,training),training$death)
    confusionMatrix(predict(modelfit,testing),testing$death)
    train_results=predict(modelfit,training,type="prob")
    test_results=predict(modelfit,testing,type="prob")
    train_results$obs=training$death
    train_results$pred=predictions_train
    test_results$obs=testing$death
    test_results$pred=predictions_test
    ROC_train<-roc(training$death,train_results[,"Death"],levels=c("Survival","Death"))
    ROC_test<-roc(testing$death,test_results[,"Death"],levels=c("Survival","Death"))
    AUC_train1[i]=ROC_train[9]$auc
    AUC_test1[i]=ROC_test[9]$auc
    cat('Completed running Model', i, '(',modellist1[i],')','of', length(modellist1),'\n')
    cat('Completed Running method:', modellist1[i], '---AUC_Train:', ROC_train[9]$auc, '---AUC_Test:', ROC_test[9]$auc, '\n')
    gc()
  }

Running method: ada 
Running method: AdaBag 
Running method: adaboost 
Running method: AdaBoost.M1 
Running method: amdai 
Running method: avNNet 
Aggregating results
Selecting tuning parameters
Fitting size = 8, decay = 0.000271, bag = FALSE on full training set


Setting direction: controls < cases

Setting direction: controls < cases



Completed running Model 6 ( avNNet ) of 187 
Completed Running method: avNNet ---AUC_Train: 0.9143373 ---AUC_Test: 0.8110236 
Running method: awnb 
Running method: awtan 
Running method: bag 


Warning message in nominalTrainWorkflow(x = x, y = y, wts = weights, info = trainInfo, :
“There were missing values in resampled performance measures.”


Aggregating results
Something is wrong; all the ROC metric values are missing:
      ROC           Sens          Spec    
 Min.   : NA   Min.   : NA   Min.   : NA  
 1st Qu.: NA   1st Qu.: NA   1st Qu.: NA  
 Median : NA   Median : NA   Median : NA  
 Mean   :NaN   Mean   :NaN   Mean   :NaN  
 3rd Qu.: NA   3rd Qu.: NA   3rd Qu.: NA  
 Max.   : NA   Max.   : NA   Max.   : NA  
 NA's   :1     NA's   :1     NA's   :1    
Running method: bagEarth 
Running method: bagEarthGCV 
Running method: bagFDA 
Running method: bagFDAGCV 
Running method: bam 
Aggregating results
Selecting tuning parameters
Fitting select = FALSE, method = ML on full training set


...

Completed running Model 172 ( svmRadialSigma ) of 187 
Completed Running method: svmRadialSigma ---AUC_Train: 1 ---AUC_Test: 0.7193987 
Running method: svmRadialWeights 
Aggregating results
Selecting tuning parameters
Fitting sigma = 0.00418, C = 0.143, Weight = 15.9 on full training set


Setting direction: controls < cases

Setting direction: controls < cases



Completed running Model 173 ( svmRadialWeights ) of 187 
Completed Running method: svmRadialWeights ---AUC_Train: 0.8427277 ---AUC_Test: 0.8196135 
Running method: svmSpectrumString 
Running method: tan 
Running method: tanSearch 
Running method: treebag 
Aggregating results
Fitting final model on full training set


Setting direction: controls < cases

Setting direction: controls < cases



Completed running Model 177 ( treebag ) of 187 
Completed Running method: treebag ---AUC_Train: 1 ---AUC_Test: 0.7863278 
Running method: vbmpRadial 
Running method: vglmAdjCat 
Running method: vglmContRatio 
Running method: vglmCumulative 
Running method: widekernelpls 
Running method: wsrf 
Running method: xgbDART 
Running method: xgbLinear 
Running method: xgbTree 
Running method: xyf 


In [ ]:
AUC1=cbind(AUC_train1,AUC_test1)
AUC1=as.data.frame(AUC1)
AUC1$Algorithms<-modellist1
print(AUC1)

    AUC_train1 AUC_test1          Algorithms
1    1.0000000 0.7888332                 ada
2    1.0000000 0.8296349              AdaBag
3    0.0000000 0.0000000            adaboost
4    1.0000000 0.7845383         AdaBoost.M1
5    0.0000000 0.0000000               amdai
6    0.8742897 0.7838225              avNNet
7    0.0000000 0.0000000                awnb
8    0.0000000 0.0000000               awtan
9    0.0000000 0.0000000                 bag
10   0.8388496 0.7752326            bagEarth
11   0.9497773 0.7136722         bagEarthGCV
12   0.9089234 0.7716535              bagFDA
13   0.9344955 0.7480315           bagFDAGCV
14   0.8714483 0.7680744                 bam
15   0.0000000 0.0000000         bartMachine
16   0.8577023 0.7845383            bayesglm
17   0.0000000 0.0000000               binda
18   0.8950238 0.8052971          blackboost
19   0.0000000 0.0000000               BstLm
20   0.0000000 0.0000000               bstSm
21   0.0000000 0.0000000             bstTree
22   0.906

In [ ]:
dim(AUC1)

[1] 100   2

In [ ]:
modellist2=model_caret_forclass$model[101:200]
modellist2
AUC_train2=array(0,length(modellist2))
AUC_test2=array(0,length(modellist2))

[1] "gamLoess"            "gamLoess"            "gamSpline"          
  [4] "gaussprLinear"       "gaussprPoly"         "gaussprPoly"        
  [7] "gaussprRadial"       "gbm"                 "gbm"                
 [10] "gbm"                 "gbm"                 "gbm_h2o"            
 [13] "gbm_h2o"             "gbm_h2o"             "gbm_h2o"            
 [16] "gbm_h2o"             "gcvEarth"            "glm"                
 [19] "glmboost"            "glmboost"            "glmnet"             
 [22] "glmnet"              "glmnet_h2o"          "glmnet_h2o"         
 [25] "glmStepAIC"          "gpls"                "hda"                
 [28] "hda"                 "hda"                 "hdda"               
 [31] "hdda"                "hdrda"               "hdrda"              
 [34] "hdrda"               "J48"                 "J48"                
 [37] "JRip"                "JRip"                "JRip"               
 [40] "kernelpls"           "kknn"                "kknn"               
 [43] "kknn"                "knn"                 "lda"                
 [46] "lda2"                "Linda"               "LMT"                
 [49] "loclda"              "logicBag"            "logicBag"           
 [52] "LogitBoost"          "logreg"              "logreg"             
 [55] "lssvmLinear"         "lssvmPoly"           "lssvmPoly"          
 [58] "lssvmPoly"           "lssvmRadial"         "lssvmRadial"        
 [61] "lvq"                 "lvq"                 "manb"               
 [64] "manb"                "mda"                 "Mlda"               
 [67] "mlp"                 "mlpKerasDecay"       "mlpKerasDecay"      
 [70] "mlpKerasDecay"       "mlpKerasDecay"       "mlpKerasDecay"      
 [73] "mlpKerasDecay"       "mlpKerasDecay"       "mlpKerasDecayCost"  
 [76] "mlpKerasDecayCost"   "mlpKerasDecayCost"   "mlpKerasDecayCost"  
 [79] "mlpKerasDecayCost"   "mlpKerasDecayCost"   "mlpKerasDecayCost"  
 [82] "mlpKerasDecayCost"   "mlpKerasDropout"     "mlpKerasDropout"    
 [85] "mlpKerasDropout"     "mlpKerasDropout"     "mlpKerasDropout"    
 [88] "mlpKerasDropout"     "mlpKerasDropout"     "mlpKerasDropoutCost"
 [91] "mlpKerasDropoutCost" "mlpKerasDropoutCost" "mlpKerasDropoutCost"
 [94] "mlpKerasDropoutCost" "mlpKerasDropoutCost" "mlpKerasDropoutCost"
 [97] "mlpKerasDropoutCost" "mlpML"               "mlpML"              
[100] "mlpML"

In [ ]:
c2 <- makeCluster(6)  # Number of cores
registerDoParallel(c2)
for (i in 1:length(modellist2)){
    skip_to_next_error <- FALSE
    cat("Running method:", modellist2[i], "\n")
    tryCatch(modelfit<-train(death~.,
                             data=training,
                             method=modellist2[i],
                             preProc=c("center","scale"),
                             metric="ROC",
                             tuneLength=3,
                             trControl=fitControl),
                             error=function(e){skip_to_next_error <<- TRUE})
    if(skip_to_next_error) {next}
    predictions_train=predict(modelfit,newdata=training)
    predictions_test=predict(modelfit,newdata=testing)
    predictions_train
    predictions_test
    confusionMatrix(predict(modelfit,training),training$death)
    confusionMatrix(predict(modelfit,testing),testing$death)
    train_results=predict(modelfit,training,type="prob")
    test_results=predict(modelfit,testing,type="prob")
    train_results$obs=training$death
    train_results$pred=predictions_train
    test_results$obs=testing$death
    test_results$pred=predictions_test
    ROC_train<-roc(training$death,train_results[,"Death"],levels=c("Survival","Death"))
    ROC_test<-roc(testing$death,test_results[,"Death"],levels=c("Survival","Death"))
    AUC_train2[i]=ROC_train[9]$auc
    AUC_test2[i]=ROC_test[9]$auc
    cat('Completed running Model', i, '(',modellist2[i],')','of', length(modellist2),'\n')
    cat('Completed Running method:', modellist2[i], '---AUC_Train:', ROC_train[9]$auc, '---AUC_Test:', ROC_test[9]$auc, '\n')
    #gc()
  }
stopCluster(c2)

Running method: gamLoess 
Running method: gamLoess 
Running method: gamSpline 
Running method: gaussprLinear 
Running method: gaussprPoly 
Running method: gaussprPoly 
Running method: gaussprRadial 
Running method: gbm 
Running method: gbm 
Running method: gbm 
Running method: gbm 
Running method: gbm_h2o 
Running method: gbm_h2o 
Running method: gbm_h2o 
Running method: gbm_h2o 
Running method: gbm_h2o 
Running method: gcvEarth 
Running method: glm 
Running method: glmboost 
Running method: glmboost 
Running method: glmnet 
Running method: glmnet 
Running method: glmnet_h2o 
Running method: glmnet_h2o 
Running method: glmStepAIC 
Running method: gpls 
Running method: hda 
Running method: hda 
Running method: hda 
Running method: hdda 
Running method: hdda 
Running method: hdrda 
Running method: hdrda 
Running method: hdrda 
Running method: J48 
Running method: J48 
Running method: JRip 
Running method: JRip 
Running method: JRip 
Running method: kernelpls 
Running method: kknn 
Runnin

Warning message in train.default(x, y, weights = w, ...):
"Class probabilities were requested for a model that does not implement them"


Running method: lssvmPoly 


Warning message in train.default(x, y, weights = w, ...):
"Class probabilities were requested for a model that does not implement them"


Running method: lssvmPoly 


...

Warning message in train.default(x, y, weights = w, ...):
"Class probabilities were requested for a model that does not implement them"


Running method: manb 
Running method: manb 
Running method: mda 
Running method: Mlda 


Warning message in train.default(x, y, weights = w, ...):
"Class probabilities were requested for a model that does not implement them"


Running method: mlp 
Running method: mlpKerasDecay 
Running method: mlpKerasDecay 
Running method: mlpKerasDecay 
Running method: mlpKerasDecay 
Running method: mlpKerasDecay 
Running method: mlpKerasDecay 
Running method: mlpKerasDecay 
Running method: mlpKerasDecayCost 
Running method: mlpKerasDecayCost 
Running method: mlpKerasDecayCost 
Running method: mlpKerasDecayCost 
Running method: mlpKerasDecayCost 
Running method: mlpKerasDecayCost 
Running method: mlpKerasDecayCost 
Running method: mlpKerasDecayCost 
Running method: mlpKerasDropout 
Running method: mlpKerasDropout 
Running method: mlpKerasDropout 
Running method: mlpKerasDropout 
Running method: mlpKerasDropout 
Running method: mlpKerasDropout 
Running method: mlpKerasDropout 
Running method: mlpKerasDropoutCost 
Running method: mlpKerasDropoutCost 
Running method: mlpKerasDropoutCost 
Running method: mlpKerasDropoutCost 
Running method: mlpKerasDropoutCost 
Running method: mlpKerasDropoutCost 
Running method: mlpKerasDropo

ERROR: Error in serialize(data, node$con): 連接寫入發生了錯誤


In [ ]:
AUC2=cbind(AUC_train2,AUC_test2)
AUC2=as.data.frame(AUC2)
AUC2$Algorithms<-modellist2
print(AUC2)

In [ ]:
modellist3=model_caret_forclass$model[201:300]
modellist3
AUC_train3=array(0,length(modellist3))
AUC_test3=array(0,length(modellist3))

In [ ]:
for (i in 1:length(modellist3)){
    skip_to_next_error <- FALSE
    cat("Running method:", modellist3[i], "\n")
    tryCatch(modelfit<-train(death~.,
                             data=training,
                             method=modellist3[i],
                             preProc=c("center","scale"),
                             metric="ROC",
                             tuneLength=3,
                             trControl=fitControl),
                             error=function(e){skip_to_next_error <<- TRUE})
    if(skip_to_next_error) {next}
    predictions_train=predict(modelfit,newdata=training)
    predictions_test=predict(modelfit,newdata=testing)
    predictions_train
    predictions_test
    confusionMatrix(predict(modelfit,training),training$death)
    confusionMatrix(predict(modelfit,testing),testing$death)
    train_results=predict(modelfit,training,type="prob")
    test_results=predict(modelfit,testing,type="prob")
    train_results$obs=training$death
    train_results$pred=predictions_train
    test_results$obs=testing$death
    test_results$pred=predictions_test
    ROC_train<-roc(training$death,train_results[,"Death"],levels=c("Survival","Death"))
    ROC_test<-roc(testing$death,test_results[,"Death"],levels=c("Survival","Death"))
    AUC_train3[i]=ROC_train[9]$auc
    AUC_test3[i]=ROC_test[9]$auc
    cat('Completed running Model', i, '(',modellist3[i],')','of', length(modellist3),'\n')
    cat('Completed Running method:', modellist3[i], '---AUC_Train:', ROC_train[9]$auc, '---AUC_Test:', ROC_test[9]$auc, '\n')
  }

In [ ]:
AUC3=cbind(AUC_train3,AUC_test3)
AUC3=as.data.frame(AUC3)
AUC3$Algorithms<-modellist3
print(AUC3)

In [ ]:
modellist4=model_caret_forclass$model[301:400]
modellist4
AUC_train4=array(0,length(modellist4))
AUC_test4=array(0,length(modellist4))

In [ ]:
for (i in 1:length(modellist4)){
    skip_to_next_error <- FALSE
    cat("Running method:", modellist4[i], "\n")
    tryCatch(modelfit<-train(death~.,
                             data=training,
                             method=modellist4[i],
                             preProc=c("center","scale"),
                             metric="ROC",
                             tuneLength=3,
                             trControl=fitControl),
                             error=function(e){skip_to_next_error <<- TRUE})
    if(skip_to_next_error) {next}
    predictions_train=predict(modelfit,newdata=training)
    predictions_test=predict(modelfit,newdata=testing)
    predictions_train
    predictions_test
    confusionMatrix(predict(modelfit,training),training$death)
    confusionMatrix(predict(modelfit,testing),testing$death)
    train_results=predict(modelfit,training,type="prob")
    test_results=predict(modelfit,testing,type="prob")
    train_results$obs=training$death
    train_results$pred=predictions_train
    test_results$obs=testing$death
    test_results$pred=predictions_test
    ROC_train<-roc(training$death,train_results[,"Death"],levels=c("Survival","Death"))
    ROC_test<-roc(testing$death,test_results[,"Death"],levels=c("Survival","Death"))
    AUC_train4[i]=ROC_train[9]$auc
    AUC_test4[i]=ROC_test[9]$auc
    cat('Completed running Model', i, '(',modellist4[i],')','of', length(modellist4),'\n')
    cat('Completed Running method:', modellist4[i], '---AUC_Train:', ROC_train[9]$auc, '---AUC_Test:', ROC_test[9]$auc, '\n')
  }

In [ ]:
AUC4=cbind(AUC_train4,AUC_test4)
AUC4=as.data.frame(AUC4)
AUC4$Algorithms<-modellist4
print(AUC4)

In [ ]:
modellist5=model_caret_forclass$model[401:nrow(model_caret_forclass)]
modellist5
AUC_train5=array(0,length(modellist5))
AUC_test5=array(0,length(modellist5))

In [ ]:
for (i in 1:length(modellist5)){
    skip_to_next_error <- FALSE
    cat("Running method:", modellist5[i], "\n")
    tryCatch(modelfit<-train(death~.,
                             data=training,
                             method=modellist5[i],
                             preProc=c("center","scale"),
                             metric="ROC",
                             tuneLength=3,
                             trControl=fitControl),
                             error=function(e){skip_to_next_error <<- TRUE})
    if(skip_to_next_error) {next}
    predictions_train=predict(modelfit,newdata=training)
    predictions_test=predict(modelfit,newdata=testing)
    predictions_train
    predictions_test
    confusionMatrix(predict(modelfit,training),training$death)
    confusionMatrix(predict(modelfit,testing),testing$death)
    train_results=predict(modelfit,training,type="prob")
    test_results=predict(modelfit,testing,type="prob")
    train_results$obs=training$death
    train_results$pred=predictions_train
    test_results$obs=testing$death
    test_results$pred=predictions_test
    ROC_train<-roc(training$death,train_results[,"Death"],levels=c("Survival","Death"))
    ROC_test<-roc(testing$death,test_results[,"Death"],levels=c("Survival","Death"))
    AUC_train5[i]=ROC_train[9]$auc
    AUC_test5[i]=ROC_test[9]$auc
    cat('Completed running Model', i, '(',modellist5[i],')','of', length(modellist5),'\n')
    cat('Completed Running method:', modellist5[i], '---AUC_Train:', ROC_train[9]$auc, '---AUC_Test:', ROC_test[9]$auc, '\n')
  }

In [ ]:
AUC5=cbind(AUC_train5,AUC_test5)
AUC5=as.data.frame(AUC5)
AUC5$Algorithms<-modellist5
print(AUC5)

In [ ]:
AUC_all=rbind(AUC1,AUC2,AUC3,AUC4,AUC5)
print(AUC_all)